In [19]:
import csv
import pprint
import pandas as pd
import pywikibot
from caphi2ipa import caphipp2ipa

# Read TSV

In [21]:
maknuune_file = "maknuune-v1.0.1.tsv"

nouns = ['NOUN:MS', 'NOUN:FS']

df = pd.read_csv(maknuune_file,sep='\t')

# Stats by PoS

In [22]:
df_count = df.groupby(['ANALYSIS'])['ANALYSIS'].count()

In [23]:
df_count.loc[lambda x : x > 50].sort_values(ascending=False)

ANALYSIS
VERB:I             6345
VERB:C             6329
VERB:P             6191
NOUN:MS            4183
NOUN:FS            3050
ADJ:MS             2388
NOUN:PHRASE        2343
NOUN:P             1894
NOUN_ACT:MS         759
VERB:PHRASE         639
NOUN_PASS           305
ADJ:P               288
ADJ:PHRASE          249
NOUN_ACT:PHRASE     117
ADJ/NOUN            108
INTERJ              106
ADJ:FS               78
ADJ_COMP             77
ADV                  71
NOUN_PROP            65
Name: ANALYSIS, dtype: int64

# First: singular nouns deriving from a root and without notes 

In [37]:
# df['LEMMA_BW'] == df['FORM_BW'] to exclude weird cases
df_nouns = df.loc[(df['ROOT'] != 'NTWS') & (df['ANALYSIS'].isin(nouns)) & (df['LEMMA_BW'] == df['FORM_BW']) & (df['NOTES'].isnull())]

In [25]:
df_nouns

,ID,ROOT,ROOT_NTWS,ROOT_1,LEMMA,LEMMA_SEARCH,FORM,LEMMA_BW,FORM_BW,CAPHI++,ANALYSIS,GLOSS,GLOSS_MSA,EXAMPLE_USAGE,NOTES,SOURCE,ANNOTATOR,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1,ء.ب.د,NaN,ء,أَبَد,أبد,أَبَد,>abad,>abad,2 a b a d,NOUN:MS,eternity;forever,NaN,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN
1,2,ء.ب.ر,NaN,ء,إِبْرِة,إبرة,إِبْرِة,<iborip,<iborip,2 i b r e,NOUN:FS,needle;injection,إِبْرَة#حُقْنَة,في إِبْرِة وقعت تحت الكنب دير بالك.#أخذت ابرة ...,NaN,NaN,شهد دعباس,NaN,NaN,NaN
7,8,ء.ب.ط,NaN,ء,أَبَاط,أباط,أَبَاط,>abaAT,>abaAT,2 a b aa t.,NOUN:MS,armpit,إِبْط,ولك ليش أباطك أسود هيك؟,NaN,NaN,شهد دعباس,NaN,NaN,NaN
8,9,ء.ب.ط,NaN,ء,بَاط,باط,بَاط,baAT,baAT,b aa t.,NOUN:MS,armpit,إِبْط,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN
10,11,ء.ب.و,NaN,ء,أَب,أب,أَب,>ab,>ab,2 a b,NOUN:MS,father,أب,الله يرحمه أبوه بقى زلمة مليح وبينشد فيه الظهر,NaN,NaN,شهد دعباس,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36039,36263,ي.م.ن,NaN,ي,يَمِين,يمين,يَمِين,yamiyn,yamiyn,y a m ii n,NOUN:MS,right,يمين,روتش لك شوي لليمين,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36040,36264,ي.م.ن,NaN,ي,يَمِين,يمين,يَمِين,yamiyn,yamiyn,y a m ii n,NOUN:MS,vow;oath,NaN,احلف يَمِين إِنه مالكاش دخَّل!,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36044,36268,ي.م.ن,NaN,ي,يُمُن,يمن,يُمُن,yumun,yumun,y u m u n,NOUN:MS,blessing,نِعْمَة,ألف مبارك الشهر وكل عام وانتو بخير وان شاء الل...,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36056,36280,ي.و.م,NaN,ي,يَومِيِّة,يومية,يَومِيِّة,yawmiy~ip,yawmiy~ip,y oo m i y y e,NOUN:FS,daily_payment,دفع بنظام المياومة,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN


# Wiktionary format

In [26]:
beginning_file = """'''User:A455bcd9/Maknuune/{entry}'''
{{{{-start-}}}}"""

template = """==South Levantine Arabic==
{{{{bulk import|{{{{R:ajp:Maknuune}}}}}}}}
{{{{ajp-root|{root}}}}}

===Pronunciation===

* {{{{ajp-IPA|{ipa}}}}}

===Noun===
{{{{ajp-noun|head={head}|g={gender}|tr={transliteration}}}}}

# [[{gloss}]]{example}

===References===
* {{{{R:ajp:Maknuune}}}}"""

end_file="{{{{-stop-}}}}"

In [27]:
site = pywikibot.Site()

# Select 10 random entries

In [11]:
# df_nouns = df_nouns.sample(n=10)

# CAPHI++ to Wikt

In [28]:
for index, row in df_nouns.iterrows():
    caphipp = row['CAPHI++'].strip()
    ipa = caphipp2ipa(caphipp, mode='wikt')
    print(row['LEMMA'], '=>', caphipp, '=>', ipa)
    
# test m a w J uu d => mawjūd (موجود)
# maw(dʒ)uːd // مَوْ ُجود
print(caphipp2ipa('m a w J uu d', mode='wikt'))
# 2 i sh t aa Q => ištāʔ (اشتاق)
# ʔiʃtaː(q)
print(caphipp2ipa('2 i sh t aa Q', mode='wikt'))

أَبَد => 2 a b a d => ʔabad
إِبْرِة => 2 i b r e => ʔibre
أَبَاط => 2 a b aa t. => ʔabāṭ
بَاط => b aa t. => bāṭ
أَب => 2 a b => ʔab
بَابَا => b aa b a => bāba
يَابَا => y aa b a => jāba
أَبُو => 2 a b u => ʔabu
تَأْثِيث => t a 2 th ii th => taʔṯīṯ
أَثَر => 2 a S a r => ʔasar
تَأْثِير => t a 2 S ii r => taʔsīr
أُجْرَة => 2 u J r a => ʔujra
إِيجَار => 2 i J aa r => ʔijār
أَجَار => 2 a J aa r => ʔajār
تَأْجِير => t a 2 J ii r => taʔjīr
أَجِر => 2 a J i r => ʔajir
أَجَل => 2 a J a l => ʔajal
تَأْجِيل => t a 2 J ii l => taʔjīl
أَخِذ => 2 a kh i D => ʔaḵid
مَأْخَذ => m a 2 kh a Z => maʔḵaz
أَخْذ => 2 a kh D => ʔaḵd
مْوَاخَذِة => m w aa kh a Z e => mwāḵaze
مُؤَاخَذِة => m u 2 aa kh a Z e => muʔāḵaze
آخْرِة => 2 aa kh r e => ʔāḵre
آخْرِة => 2 aa kh r e => ʔāḵre
تَأْخِير => t a 2 kh ii r => taʔḵīr
آخُور => 2 aa kh uu r => ʔāḵūr
مُؤَخَّر => m u 2 a kh kh a r => muʔaḵḵar
أُخُت => 2 u kh u t => ʔuḵut
مْخَاوَاة => m kh aa w aa => mḵāwā
أَخ => 2 a kh => ʔaḵ
خَيّ => kh a y y => ḵajj
مُآخَاة => m u 2 

تَبْوِيس => t a b w ii s => tabwīs
بَوش => b oo sh => bōš
بَاطْيِة => b aa t. y e => bāṭje
بَوَط => b a w a t. => bawaṭ
بُومِة => b uu m e => būme
بَوّ => b a w w => baww
بِيئَة => b ii 2 a => bīʔa
بِيت => b ii t => bīt
بَيت => b ee t => bēt
بَيت => b ee t => bēt
مَبِيت => m a b ii t => mabīt
بَيَات => b a y aa t => bajāt
بَيتُون => b ee t uu n => bētūn
بَيج => b ee J => bēj
بَيَاخَة => b a y aa kh a => bajāḵa
اِبَادِة => 2 i b aa d e => ʔibāde
مُبِيد => m u b ii d => mubīd
بَيَّارَة => b a y y aa r a => bajjāra
مُبَيِّض => m u b a y y i D. => mubajjiḍ
تَبْيِيض => t a b ii D. => tabīḍ
بَيَاض => b a y aa D. => bajāḍ
بَيعَة => b ee 3 a => bēʕa
بَيع => b ee 3 => bēʕ
بَيَّاع => b a y y aa 3 => bajjāʕ
بَين => b ee n => bēn
بَين => b ee n => bēn
تَأْتَأَة => t a 2 t a 2 a => taʔtaʔa
تَابُوت => t aa b uu t => tābūt
مُتَابَعَة => m u t aa b a 3 a => mutābaʕa
تَبَع => t a b a 3 => tabaʕ
مُتَابِع => m u t aa b i 3 => mutābiʕ
تَتْبِيلِة => t a l b ii l e => talbīle
تَبَّان => t a b b aa n => tabb

حِرْبَايِة => 7 i r b aa y e => ḥirbāje
مُحَارِب => m u 7 aa r i b => muḥārib
مُحَارَبِة => m u 7 aa r a b e => muḥārabe
حُرُب => 7 u r u b => ḥurub
مِحْرَاث => m i 7 r aa th => miḥrāṯ
حْرَاثِة => 7 r aa th e => ḥrāṯe
حْرَاث => 7 r aa th => ḥrāṯ
حُرْجَايِة => 7 u r dj aa y e => ḥurdjāje
إِحْرَاج => 2 i 7 r aa J => ʔiḥrāj
حُرْج => 7 u r u dj => ḥurudj
حَرْحَشِة => 7 a r 7 a sh e => ḥarḥaše
حَرَد => 7 a r a d => ḥarad
حَرَدَان => 7 a r a d aa n => ḥaradān
حَرْدَنِة => 7 a r d a n e => ḥardane
حَرْوَدِة => 7 a r w a d e => ḥarwade
حُرْدُبِة => 7 u r d u b e => ḥurdube
حُرِّيِّة => 7 u r r i y y e => ḥurrijje
حَرَارَة => 7 a r aa r a => ḥarāra
تَحْرِير => t a 7 r ii r => taḥrīr
حَرِير => 7 a r ii r => ḥarīr
حِرِز => 7 i r i z => ḥiriz
حَارِس => 7 aa r i s => ḥāris
حَرْشِة => 7 a r sh e => ḥarše
تَحَرُّش => t 7 a r r u sh => tḥarruš
مُتَحَرِّش => m u t a 7 a r r i sh => mutaḥarriš
حَرْش => 7 a r sh => ḥarš
حَرِيش => 7 a r ii sh => ḥarīš
حِرْص => 7 i r s. => ḥirṣ
تَحْرِيض => t a 7 r ii D. =>

خُشُوع => kh u sh uu 3 => ḵušūʕ
خَاشُوقَة => kh aa sh uu q||k a => khaashuuqa, khaashuuka
مُخْشَاقَة => m u kh sh aa q||k a => mukhshaaqa, mukhshaaka
خَشِم => kh a sh i m => ḵašim
تَخْصِير => t a kh s. ii r => taḵṣīr
خَصِر => kh a s. i r => ḵaṣir
اِخْتِصَار => 2 i kh t i s. aa r => ʔiḵtiṣār
خَاصِّيِّة => kh aa s. s. i y y e => ḵāṣṣijje
تَخَصُّص => t a kh a s. s. u s. => taḵaṣṣuṣ
خُصّ => kh u s. s. => ḵuṣṣ
خُصُوص => kh u s. uu s. => ḵuṣūṣ
خُصْلِة => kh u s. l e => ḵuṣle
خُصْلِة => kh u s. l e => ḵuṣle
خَصِم => kh a s. i m => ḵaṣim
خَصِم => kh a s. i m => ḵaṣim
خِصْيِة => kh i s. y e => ḵiṣje
خَصَّا => kh a s. s. a => ḵaṣṣa
خَضْخَضَة => kh a D. kh a D. a => ḵaḍḵaḍa
خُضَرْجي => kh u D. a r J i => ḵuḍarji
خَضَار => kh a D. aa r => ḵaḍār
خُضَار => kh u D. aa r => ḵuḍār
خُضْرَة => kh u D. r a => ḵuḍra
خَضَّة => kh a D. D. a => ḵaḍḍa
خَطِيِّة => kh a t. i y y e => ḵaṭijje
خَطَأ => kh a t. a 2 => ḵaṭaʔ
خَطِيب => kh a t. ii b => ḵaṭīb
خَطِيب => kh a t. ii b => ḵaṭīb
خِطَاب => kh i t. aa b => ḵi

مِتْرَبِّي => m i t r a b b i => mitrabbi
مَرْتَبِة => m a r t a b e => martabe
تَرْتِيب => t a r t ii b => tartīb
رَاتِب => r aa t i b => rātib
رُتْبِة => r u t b e => rutbe
مَرْتَبَان => m a r t a b aa n => martabān
اِرْتِجَاج => 2 i r t i J aa J => ʔirtijāj
مُرْجَيحَة => m u r J ee 7 a => murjēḥa
مْرَاجَدِة => m r aa J a d e => mrājade
رَجْعَة => r a J 3 a => rajʕa
مْرَاجَعَة => m r aa J a 3 a => mrājaʕa
مُرَاجَعَة => m u r aa J a 3 a => murājaʕa
تَرْجِيع => t a r J ii 3 => tarjīʕ
اِسْتِرْجَاع => 2 i s t i r J aa 3 => ʔistirjāʕ
رَجْفِة => r a J f e => rajfe
إِجِر => 2 i J i r => ʔijir
مَرْجَلِة => m a r J a l e => marjale
اِجِر => 2 i J i r => ʔijir
رُجُولِة => r u J uu l e => rujūle
رَجُل => r a J u l => rajul
رِجَّال => r i J J aa l => rijjāl
اِسْتِرْجَال => 2 i s t i r J aa l => ʔistirjāl
اِرْتِجَال => 2 i r t i J aa l => ʔirtijāl
رِجِل => r i J i l => rijil
رَجِم => r a J i m => rajim
رَجَاء => r a J aa 2 => rajāʔ
رَجَا => r a J a => raja
رِحْلِة => r i 7 l e => riḥle
مَرْحَلِة 

...

عِنَاق => 3 i n aa q => ʕināq
اِعْتِنَاق => 2 i 3 t i n aa q => ʔiʕtināq
عُنُق => 3 u n u q => ʕunuq
عَنْقُود => 3 a n q uu d => ʕanqūd
عَنْقَرَة => 3 a n q a r a => ʕanqara
عِنْوَان => 3 i n w aa n => ʕinwān
مَعْنَى => m a 3 n a => maʕna
عِنَايِة => 3 i n aa y e => ʕināje
مُعَانَاة => m u 3 aa n aa => muʕānā
عُهْدِة => 3 u h d a => ʕuhda
عَهِد => 3 a h i d => ʕahid
تَعَهُّد => t a 3 a h h u d => taʕahhud
اِنْعِوَاج => 2 i n 3 i w aa J => ʔinʕiwāj
عَوَج => 3 a w a J => ʕawaj
عَودِة => 3 oo d e => ʕōde
عَادِة => 3 aa d e => ʕāde
عَائِد => 3 aa 2 i d => ʕāʔid
تَعْوِيد => t a 3 w ii d => taʕwīd
عِيَاذ => 3 i y aa Z => ʕijāz
تَعْوِيذِة => t a 3 w ii Z e => taʕwīze
عَوَر => 3 a w a r => ʕawar
عْوَار => 3 w aa r => ʕwār
عَوْرَة => 3 a w r a => ʕawra
عَوْرَة => 3 a w r a => ʕawra
عَازِة => 3 aa z e => ʕāze
عَوَض => 3 a w a D. => ʕawaḍ
تَعْوِيض => t a 3 w ii D. => taʕwīḍ
عَوفِة => 3 oo f e => ʕōfe
إِعَاقَة => 2 i 3 aa q a => ʔiʕāqa
مُعِيق => m u 3 ii q => muʕīq
عَائِق => 3 aa 2 i q => ʕāʔiq
عَ

تَفَوُّق => t a f a w w u q => tafawwuq
فَيّ => f a y y => fajj
مَفَاد => m a f aa d => mafād
فَائِدِة => f aa 2 i d e => fāʔide
فَايْدِة => f aa y d e => fājde
فَود => f oo d => fōd
فِيد => f ii d => fīd
فَيد => f ee d => fēd
فِيش => f ii sh => fīš
فِيشِة => f ii sh e => fīše
فَيض => f ee D. => fēḍ
فَيَضَان => f a y a D. aa n => fajaḍān
فَيَاعَة => f aa y aa 3 a => fājāʕa
فِيل => f ii l => fīl
قَبِّة => Q a b b e => ʔabbe
قُبِّة => q u b b e => qubbe
قَبِّيِّة => q a b b i y y e => qabbijje
قَابِة => q aa b e => qābe
قُبِح => q u b u 7 => qubuḥ
تَقْبِيح => t a q b ii 7 => taqbīḥ
قَبِر => Q a b i r => ʔabir
اِقْتِبَاس => 2 i q t i b aa s => ʔiqtibās
تَقْبِيص => t a q b ii s. => taqbīṣ
قَبْضَة => Q a b D. a => ʔabḍa
قَبِض => Q a b i d. => ʔabiḍ
قُبْعَة => Q u b 3 a => ʔubʕa
قُبُع => q u b u 3 => qubuʕ
قُبُع => q u b u 3 => qubuʕ
قُبْقَاب => q u b q aa b , 2 u b 2 aa b => qubqāb, ʔubʔāb
مْقَابِيل => m q aa b ii l => mqābīl
مُقَابِل => m u q aa b i l => muqābil
قَبِل => Q a b i l => ʔabil

كَرْفَعَة => K a r f a 3 a => karfaʕa
كْرَيك => k r ee k => krēk
كْرَيك => k r ee k => krēk
كَرْكُوبِة => k a r k uu b e => karkūbe
كِرْكَفِّة => tsh i r tsh a f f e => tširtšaffe
كُرْكُفِّة => K u r K u f f e => kurkuffe
كُرْكُم => k u r k u m => kurkum
كُرْكَمِّة => k u r k a m m e => kurkamme
كَرَامِة => k a r aa m e => karāme
كَرْمِيِّة => k a r m i y y e => karmijje
إِكْرَامِيِّة => 2 i k r aa m i y y e => ʔikrāmijje
كَرَم => k a r a m => karam
كَرِم => k a r i m => karim
تَكْرِيم => t a k r ii m => takrīm
كِرْمَال => k i r m aa l => kirmāl
كَرْمَشِة => k a r m a sh e => karmaše
كَرْنَبِة => K a r n a b e => karnabe
كَرَاهيِّة => k a r aa h i y y e => karāhijje
كُرُه => K u r u h => kuruh
إِكْرَاه => 2 i k r aa h => ʔikrāh
كَورَة => k oo r a => kōra
كُرَة => k u r a => kura
كَرْوَتِة => k a r w a t e => karwate
كُزْبَرَة => k u z b a r a => kuzbara
كِزْبَرَة => k i z b a r a => kizbara
كَزْدَرَة => k a z d a r a => kazdara
كُزْلُك => k u z l u k => kuzluk
كُزْلُك => tsh u z l u ts

مِشْط => m i sh t. => mišṭ
مْشَاط => m sh aa t. => mšāṭ
تَمْشِيط => t a m sh ii t. => tamšīṭ
مَشِي => m a sh i => maši
مَشَّاي => m a sh sh aa y => maššāj
مَصَّاصَة => m a s. s aa s. a => maṣsāṣa
مَصَّاصَة => m a s. s aa s. a => maṣsāṣa
مَص => m a s. s => maṣs
مَصِّيص => m a s. s. ii s. => maṣṣīṣ
مَصِل => m a s. i l => maṣil
مَصْمَصَة => m a s. m a s. a => maṣmaṣa
مَضِغ => m a D. i gh => maḍiḡ
مَضْمَضَة => m a D. m a D. a => maḍmaḍa
تِمْضَايِة => t i m D. aa y i t => timḍājit
مَاضِي => m aa D. i => māḍi
مَطَر => m a t. a r => maṭar
مَطَرَة => m a t. a r a => maṭara
مَطَرَة => m a t. a r a => maṭara
مَطَارَة => m a t. aa r a => maṭāra
مَطَّة => m a t. t. a => maṭṭa
مَطَّاط => m a t. t. aa t. => maṭṭāṭ
مُمَاطَلِة => m u m aa t. a l e => mumāṭale
مَطْمَطَة => m a t. m a t. a => maṭmaṭa
مَطْمَط => m a t. m a t. => maṭmaṭ
مِعْدِة => m i 3 d e => miʕde
مَعَّاطَة => m a 3 3 aa t. a => maʕʕāṭa
مَعَّاكِة => m a 3 3 aa K K e => maʕʕākke
مَعِك => m a 3 i K => maʕik
تَمَعُّن => t a m a 3 3 u n => 

نَهْنَهَة => n a h n a h a => nahnaha
نِهَايِة => n i h aa y e => nihāje
إِنْهَاء => 2 i n h aa 2 => ʔinhāʔ
نِيَابِة => n i y aa b e => nijābe
مُنَاوَبِة => m u n aa w a b e => munāwabe
تَنَاوُب => t a n aa w u b => tanāwub
نَوبِة => n oo b e => nōbe
نَائِب => n aa 2 i b => nāʔib
مَنَاحَة => m a n aa 7 a => manāḥa
نَوَّاحَة => n a w w aa 7 a => nawwāḥa
نْيَاح => n w aa 7 => nwāḥ
نْوَاح => n w aa 7 => nwāḥ
نَوْرَنِة => n a w r a n e => nawrane
نُوَّار => n u w w aa r => nuwwār
نَار => n aa r => nār
نُور => n uu r => nūr
إِنَارَة => 2 i n aa r a => ʔināra
نَوَّاسِة => n a w w aa s e => nawwāse
مُنَاوَشِة => m u n aa w a sh e => munāwaše
تَنْوِيص => t a n w ii s. => tanwīṣ
تَنْوِيع => t a n w ii 3 => tanwīʕ
نَوع => n oo 3 => nōʕ
نَول => n oo l => nōl
مِنوَال => m i n w aa l => minwāl
نَومِة => n oo m e => nōme
مَنَام => m a n aa m => manām
نَوم => n oo m => nōm
نُونِيِّة => n uu n i y y e => nūnijje
تَنْوِيه => t a n w ii h => tanwīh
نِيِّة => n i y y e => nijje
إِنَابِة => 2 i n aa b e =

# Test entries

In [31]:
test_entries = [
    'أب',
    'أباط',
    'أجار',
    'أجرة',
    'أخذ',
    'ألفة',
    'إيجار',
    'باط',
    'بلع',
    'تأثير',
    'تأجير',
    'تأجيل',
    'تجارة',
    'خرقة',
    'زهاب',
    'عنصرية',
    'غواصة',
    'قذلة',
    'كماجة',
    'لقمة',
    'مهاجر',
    'مواخذة',
]

# Save entries in Wiktionary

In [39]:
i = 0
for index, row in df_nouns.iterrows():
    # remove _[auto] from entries generated automatically from other entries
    gloss = row['GLOSS'].replace('_[auto]', '').strip()
    # filter out more complex glosses
    if ';' not in gloss and '_' not in gloss and ' ' not in gloss:
        entry_values = {}
        entry_values['entry'] = row['LEMMA_SEARCH'].strip()
        entry_values['root'] = row['ROOT'].strip().replace('.', ' ')
        entry_values['head'] = row['FORM'].strip()
        entry_values['gender'] = 'f' if row['ANALYSIS'] == 'NOUN:FS' else 'm'

        caphipp = row['CAPHI++'].strip()
        entry_values['transliteration'] = caphipp2ipa(caphipp, mode='wikt')
        entry_values['ipa'] = entry_values['transliteration']

        entry_values['gloss'] = row['GLOSS']
        entry_values['example'] = ''
        if not pd.isnull(row['EXAMPLE_USAGE']):
            example = entry_values['example'].strip()
            # Fill example if not null
            if example != '':
                entry_values['example'] = '\n#: {{{{ux|ajp|{example}}}}}'.format(example=example)
        
        new_entry = template.format(**entry_values)
        
        page_name = u"User:A455bcd9/Maknuune/{entry}".format(entry=entry_values['entry'])
        page = pywikibot.Page(site, page_name)
        if entry_values['entry'] in test_entries:
            page.text = new_entry
            print(new_entry)
            page.save(u"Updating entries")
            # i += 1
        
        if i > 10:
            break

print('end')

==South Levantine Arabic==
{{bulk import|{{R:ajp:Maknuune}}}}
{{ajp-root|ء ل ف}}

===Pronunciation===

* {{ajp-IPA|ʔulfe}}

===Noun===
{{ajp-noun|head=أُلْفِة|g=f|tr=ʔulfe}}

# [[affinity]]

===References===
* {{R:ajp:Maknuune}}


Page [[User:A455bcd9/Maknuune/ألفة]] saved


==South Levantine Arabic==
{{bulk import|{{R:ajp:Maknuune}}}}
{{ajp-root|خ ر ق}}

===Pronunciation===

* {{ajp-IPA|ḵirka}}

===Noun===
{{ajp-noun|head=خِرْقَة|g=f|tr=ḵirka}}

# [[headscarf]]

===References===
* {{R:ajp:Maknuune}}


Sleeping for 8.9 seconds, 2023-01-09 17:56:07
Page [[User:A455bcd9/Maknuune/خرقة]] saved


==South Levantine Arabic==
{{bulk import|{{R:ajp:Maknuune}}}}
{{ajp-root|غ و ص}}

===Pronunciation===

* {{ajp-IPA|ḡawwāṣa}}

===Noun===
{{ajp-noun|head=غَوَّاصَة|g=f|tr=ḡawwāṣa}}

# [[submarine]]

===References===
* {{R:ajp:Maknuune}}


Sleeping for 8.8 seconds, 2023-01-09 17:56:17
Page [[User:A455bcd9/Maknuune/غواصة]] saved


==South Levantine Arabic==
{{bulk import|{{R:ajp:Maknuune}}}}
{{ajp-root|ل ق م}}

===Pronunciation===

* {{ajp-IPA|luʔme}}

===Noun===
{{ajp-noun|head=لُقْمِة|g=f|tr=luʔme}}

# [[bite]]

===References===
* {{R:ajp:Maknuune}}


Sleeping for 8.9 seconds, 2023-01-09 17:56:27


end


Page [[User:A455bcd9/Maknuune/لقمة]] saved


In [ ]:
# zhāb زْهَاب
# قُذْلِة quḏle
# 